# Introduction

In early 2017, Quora released a really interesting [dataset](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) on question pairs. Basically, one of the most important jobs for Quora is identifying when two questions are asking the same thing. For example "" and "" have basically the same meaning. This is important for Quora to recognize because they don't want 3 of the same questions, each with different answers. 

In this notebook, we'll look at the dataset that Quora released, as well as creating a machine learning model that determines whether two questions can be considered pairs or not. 